In [ ]:
!sudo apt install tesseract-ocr -y

In [ ]:
!pip install cairosvg pytesseract

In [ ]:
!pip install openai

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import re
from urllib.request import urlopen

import requests

import os

import matplotlib.pyplot as plt

from pathlib import Path

import cairosvg
from PIL import Image
from io import BytesIO

import pytesseract

import spacy
from unicodedata import normalize

import cv2

import openai

In [ ]:
!unzip /content/drive/MyDrive/Deceptive-Research/annotation.zip

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

### Cleaning the Data

In [ ]:
def clean_text(text):

    text = normalize("NFKD", text) #Normalization

    text = re.sub(r"[^\w\s]","", text) #Remove Punc

    # text = " ".join([token.lemma_ for token in nlp(text) if not token.is_stop])

    text = re.sub("\s+", " ", text)

    text = text.strip()

    return text

### Cleaning the image

In [ ]:
def clean_img(img):

    img = cv2.resize(img, (224, 224))

    # Normalization
    img = img/255.0

    return img

### Importing Open API Key

In [ ]:
with open("/content/openai_api_key.txt", "r") as f:
  openai.api_key = f.read()

### ChatGPT Connection

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

### Summarize the webpage using ChatGPT

In [ ]:
def summarize_webpage(website_link):

  prompt = f"""
  Your task is to summarize the website.

  Website: ```{website_link}```
  """
  response = get_completion(prompt)

  return response

### OCR - Image to Text

In [ ]:
def get_data(image):

    txt = pytesseract.image_to_string(image, lang="eng")
    txt = re.sub("[\n]{2,}", "\t\t", txt)
    txt = re.sub("\n", "", txt)
    txt = re.sub("\t\t", "\n", txt)

    if not txt:
      txt = "No Information"

    return txt

### Cosine Similarity

In [ ]:
def cos_similarity(text1, text2):

  doc1 = nlp(text1)
  doc2 = nlp(text2)

  return doc1.similarity(doc2)

### Annotated Deceptive Data Extrcation

In [ ]:
def extract_data(soup):

  imgs = []
  img_txts = []
  y_true = []
  cos_sims = []


  text = summarize_webpage(soup)

  # find all images in URL
  images = soup.findAll('img', alt=True)

  # checking if images is not zero
  if len(images) != 0:
    for i, image in enumerate(images):

      try:
        image_link = image["data-srcset"]

      except:
        try:
      
          image_link = image["data-src"]

        except:
          try:

            image_link = image["data-fallback-src"]
          except:
            try:
          
              image_link = image["src"]

            except Exception as e:
              print(f"Error: {e}")

      try:

        alt_text = image["alt"]

        if alt_text in ["deceptive", "normal"]:

          y = 1 if alt_text == "deceptive" else 0

          if Path(image_link).suffix ==  ".svg":
            img_png = cairosvg.svg2png(url = image_link)
            img = plt.imread(BytesIO(img_png))[:,:,:3]
            img = np.array(Image.fromarray((img * 255).astype(np.uint8)))
            
          else: img = plt.imread(image_link)
          
          img_txt = get_data(img)
          img = clean_img(img)
          cos_sim = cos_similarity(text, img_txt)

          imgs.append(img)
          img_txts.append(img_txt)
          cos_sims.append(cos_sim)
          y_true.append(y)
            
      except Exception as e:
        print(f"Error: {e}")

  return imgs, img_txts, cos_sims, y_true

In [ ]:
def main(f):

  with open(f, "rb") as f:
    html = f.read().decode('utf-8')
  
  soup = BeautifulSoup(html, 'html.parser')

  # Call folder create function
  return extract_data(soup)

In [ ]:
os.chdir("/content/annotation")

X_img = []
X_txt = []
X_cos = []
y = []

for f in list(Path('/content/annotation').glob('*.html')):
  img, txt, cos_sim, label = main(f)
  X_img.extend(img)
  X_txt.extend(txt)
  X_cos.extend(cos_sim)
  y.extend(label)

<ipython-input-8-ea8c808ccdcd>:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  return doc1.similarity(doc2)
<ipython-input-8-ea8c808ccdcd>:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models inste

Error: Cannot handle this data type: (1, 1, 3), <f4


<ipython-input-8-ea8c808ccdcd>:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  return doc1.similarity(doc2)


In [ ]:
len(y)

5

In [ ]:
X_cos

[-0.03491137395281285,
 -0.1219453025589311,
 0.025956913693130596,
 0.09302007141181075,
 0.15937664581111233]

### Extras

In [ ]:
with open("/content/annotation/What is The Rock Workout Routine_ - SET FOR SET.html", "r") as f:
  html = f.read()

soup = BeautifulSoup(html, 'html.parser')

for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())

# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))

# drop blank lines
text = ' '.join(chunk for chunk in chunks if chunk)

print(text)